# **CHAPTER 10**
# **Data Aggregation and Group Operations**

## **10.1 GroupBy Mechanics**

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

In [2]:
df

,key1,key2,data1,data2
0,a,one,1.073885,0.905585
1,a,two,0.461267,0.221634
2,b,one,-0.698703,0.214824
3,b,two,-1.971444,1.120113
4,a,one,-0.266426,-0.369796


In [3]:
grouped = df['data1'].groupby(df['key1'])

In [4]:
 grouped

In [5]:
grouped.mean()

key1
a    0.422908
b   -1.335073
Name: data1, dtype: float64

In [6]:
 means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [7]:
means

key1  key2
a     one     0.403729
      two     0.461267
b     one    -0.698703
      two    -1.971444
Name: data1, dtype: float64

In [8]:
means.unstack()

key2,one,two
key1,,
a,0.403729,0.461267
b,-0.698703,-1.971444


In [9]:
 states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [10]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [11]:
df['data1'].groupby([states, years]).mean()

California  2005    0.461267
            2006   -0.698703
Ohio        2005   -0.448779
            2006   -0.266426
Name: data1, dtype: float64

In [12]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.422908,0.252475
b,-1.335073,0.667468


In [13]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.403729  0.267895
     two   0.461267  0.221634
b    one  -0.698703  0.214824
     two  -1.971444  1.120113

In [14]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### **Iterating Over Groups**

In [15]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.073885  0.905585
1    a  two  0.461267  0.221634
4    a  one -0.266426 -0.369796
b
  key1 key2     data1     data2
2    b  one -0.698703  0.214824
3    b  two -1.971444  1.120113


In [16]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.073885  0.905585
4    a  one -0.266426 -0.369796
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.461267  0.221634
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.698703  0.214824
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.971444  1.120113


In [17]:
pieces = dict(list(df.groupby('key1')))

In [18]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.698703,0.214824
3,b,two,-1.971444,1.120113


In [19]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [20]:
grouped = df.groupby(df.dtypes, axis=1)

In [21]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.073885  0.905585
1  0.461267  0.221634
2 -0.698703  0.214824
3 -1.971444  1.120113
4 -0.266426 -0.369796
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### **Selecting a Column or Subset of Columns**

In [22]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

In [23]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [24]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.267895
     two   0.221634
b    one   0.214824
     two   1.120113

In [25]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [26]:
s_grouped

In [27]:
s_grouped.mean()

key1  key2
a     one     0.267895
      two     0.221634
b     one     0.214824
      two     1.120113
Name: data2, dtype: float64

### **Grouping with Dicts and Series**

In [28]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [29]:
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values

In [30]:
people

,a,b,c,d,e
Joe,1.083654,-1.056828,-0.536243,-1.004354,-0.430948
Steve,-0.830170,0.612740,0.334183,-1.967403,-0.580281
Wes,-0.568662,NaN,NaN,2.267520,-0.930557
Jim,-1.151946,0.634966,0.152873,0.545358,0.989068
Travis,-0.290441,-1.742894,-0.107544,1.321065,-1.942269


In [31]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [32]:
by_column = people.groupby(mapping, axis=1)

In [33]:
 by_column.sum()

,blue,red
Joe,-1.540597,-0.404122
Steve,-1.633220,-0.797711
Wes,2.267520,-1.499219
Jim,0.698231,0.472088
Travis,1.213520,-3.975604


In [34]:
map_series = pd.Series(mapping)

In [35]:
 map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [36]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### **Grouping with Functions**

In [37]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.636954,-0.421862,-0.383370,1.808523,-0.372436
5,-0.830170,0.612740,0.334183,-1.967403,-0.580281
6,-0.290441,-1.742894,-0.107544,1.321065,-1.942269


In [38]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [39]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.568662 -1.056828 -0.536243 -1.004354 -0.930557
  two -1.151946  0.634966  0.152873  0.545358  0.989068
5 one -0.830170  0.612740  0.334183 -1.967403 -0.580281
6 two -0.290441 -1.742894 -0.107544  1.321065 -1.942269

### **Grouping by Index Levels**

In [40]:
 columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                      [1, 3, 5, 1, 3]],
                                     names=['cty', 'tenor'])

In [41]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

In [42]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.125424 -0.135137  0.019196 -0.820619  0.808266
1     -0.562468 -0.523100  0.300439 -1.056444  0.897381
2      0.217570 -1.211784  1.248307  0.155112  0.254955
3      1.285111 -0.562131 -0.860484 -0.890173  1.030268

In [43]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## **10.2 Data Aggregation**

In [44]:
df

,key1,key2,data1,data2
0,a,one,1.073885,0.905585
1,a,two,0.461267,0.221634
2,b,one,-0.698703,0.214824
3,b,two,-1.971444,1.120113
4,a,one,-0.266426,-0.369796


In [45]:
grouped = df.groupby('key1')

In [46]:
grouped['data1'].quantile(0.9)

key1
a    0.951361
b   -0.825977
Name: data1, dtype: float64

In [47]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [48]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.340311,1.275381
b,1.272741,0.905289


In [49]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.422908  0.670978 -0.266426  0.097420  0.461267  0.767576   
b      2.0 -1.335073  0.899964 -1.971444 -1.653259 -1.335073 -1.016888   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.073885   3.0  0.252475  0.638250 -0.369796 -0.074081  0.221634   
b    -0.698703   2.0  0.667468  0.640136  0.214824  0.441146  0.667468   

                         
          75%       max  
key1                     
a     0.56361  0.905585  
b     0.89379  1.120113

### **Column-Wise and Multiple Function Application**

In [50]:
tips = pd.read_csv('../examples/tips.csv')

In [51]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [52]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [53]:
grouped = tips.groupby(['day', 'smoker'])

In [54]:
grouped_pct = grouped['tip_pct']

In [55]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [56]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [57]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [58]:
functions = ['count', 'mean', 'max']

In [59]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

<ipython-input-59-426117f1e146>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


In [60]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [61]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [62]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [63]:
 grouped['tip_pct', 'total_bill'].agg(ftuples)

<ipython-input-63-c2b2ffd79840>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [64]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [65]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### **Returning Aggregated Data Without Row Indexes**

In [66]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## **10.3 Apply: General split-apply-combine**

In [67]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [69]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [70]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [71]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [80]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [81]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [82]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [83]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### **Suppressing the Group Keys**

In [84]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### **Quantile and Bucket Analysis**

In [85]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

In [88]:
frame

,data1,data2
0,1.618285,1.676823
1,0.848864,-0.648728
2,1.683807,0.406366
3,1.715645,-0.720924
4,0.459396,-0.514582
...,...,...
995,-0.747671,2.014821
996,-0.367998,-0.404429
997,-0.549790,-1.728573
998,-0.330065,-0.895593


In [86]:
quartiles = pd.cut(frame.data1, 4)# 4-> bölme ölçütü

In [87]:
quartiles[:10]

0       (1.57, 3.319]
1      (-0.179, 1.57]
2       (1.57, 3.319]
3       (1.57, 3.319]
4      (-0.179, 1.57]
5    (-1.928, -0.179]
6      (-0.179, 1.57]
7    (-3.684, -1.928]
8      (-0.179, 1.57]
9      (-0.179, 1.57]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.684, -1.928] < (-1.928, -0.179] < (-0.179, 1.57] < (1.57, 3.319]]

In [89]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

In [90]:
grouped = frame.data2.groupby(quartiles)

In [91]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.684, -1.928]",-2.374568,2.212458,25.0,-0.086951
"(-1.928, -0.179]",-2.665505,2.934642,409.0,0.060584
"(-0.179, 1.57]",-2.582146,2.889752,498.0,0.021827
"(1.57, 3.319]",-2.082683,2.459169,68.0,0.062126


In [92]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.374568,2.934642,100.0,0.148709
1,-1.839320,2.116021,100.0,0.092604
2,-2.236847,2.638976,100.0,-0.106961
3,-2.665505,2.676437,100.0,0.034434
4,-1.759901,2.889752,100.0,0.078490
5,-2.341361,2.814054,100.0,-0.016362
6,-1.998330,2.368435,100.0,-0.021095
7,-2.413504,2.124517,100.0,0.196155
8,-2.582146,2.480127,100.0,0.014367


### **Example: Filling Missing Values with Group-Specific Values**

In [93]:
s=pd.Series(np.random.randn(6))

In [94]:
s

0   -2.295496
1   -1.395534
2    1.099386
3    0.478714
4    0.932058
5    0.633902
dtype: float64

In [101]:
s[::2]=np.nan

In [102]:
s

0         NaN
1   -1.395534
2         NaN
3    0.478714
4         NaN
5    0.633902
dtype: float64

In [103]:
s.fillna(s.mean())
#NA / NaN değerlerini belirtilen yöntemi kullanarak doldurur.

0   -0.094306
1   -1.395534
2   -0.094306
3    0.478714
4   -0.094306
5    0.633902
dtype: float64

In [110]:
states=['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']

In [111]:
group_key=['East']*4+['West']*4

In [112]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [113]:
data=pd.Series(np.random.randn(8),index=states)

In [114]:
data

Ohio          0.381704
New York     -0.675649
Vermont       0.406539
Florida       0.590021
Oregon       -0.778736
Nevada        0.563306
California    0.735220
Idaho        -2.721133
dtype: float64

In [115]:
data[['Vermont','Nevada','Idaho']]=np.nan

In [116]:
data

Ohio          0.381704
New York     -0.675649
Vermont            NaN
Florida       0.590021
Oregon       -0.778736
Nevada             NaN
California    0.735220
Idaho              NaN
dtype: float64

In [122]:
data.groupby(group_key).mean()

East    0.098692
West   -0.021758
dtype: float64

Groupby işlemi, nesneyi bölme, bir işlev uygulama ve sonuçları birleştirmenin bazı kombinasyonlarını içerir. Bu, büyük miktarda veriyi gruplamak ve bu gruplar üzerindeki işlemleri hesaplamak için kullanılabilir.

In [123]:
fill_mean=lambda g: g.fillna(g.mean())

In [124]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.381704
New York     -0.675649
Vermont       0.098692
Florida       0.590021
Oregon       -0.778736
Nevada       -0.021758
California    0.735220
Idaho        -0.021758
dtype: float64

In [125]:
fill_values = {'East': 0.5, 'West': -1}

In [126]:
fill_func=lambda g: g.fillna(fill_values[g.name])

In [127]:
data.groupby(group_key).apply(fill_func)

Ohio          0.381704
New York     -0.675649
Vermont       0.500000
Florida       0.590021
Oregon       -0.778736
Nevada       -1.000000
California    0.735220
Idaho        -1.000000
dtype: float64

### **Example: Random Sampling and Permutation**

In [128]:
# Hearts, Spades, Clubs, Diamonds
suits=['H','S','C','D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

**Extend () yöntemi**, bir yinelenebilir öğenin tüm öğelerini (liste, tuple, dize vb.) Listenin sonuna ekler.

In [129]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [131]:
def draw(deck, n=5):
    return deck.sample(n)

**sample ()** Python'da, list, tuple, string veya set gibi diziden seçilen öğelerin belirli bir uzunluk listesini döndürendahili bir rastgele modül işlevidir. Değiştirilmeden rastgele örnekleme için kullanılır.

In [132]:
draw(deck)

9D     9
AH     1
AD     1
6D     6
QH    10
dtype: int64

In [133]:
get_suit = lambda card: card[-1] # last letter is suit

In [134]:
deck.groupby(get_suit).apply(draw, n=2)

C  5C     5
   QC    10
D  KD    10
   4D     4
H  5H     5
   JH    10
S  JS    10
   5S     5
dtype: int64

In [135]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

5C      5
QC     10
10D    10
QD     10
6H      6
2H      2
3S      3
10S    10
dtype: int64

### **Example: Group Weighted Average and Correlation**

In [137]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                                               'b', 'b', 'b', 'b'],
                                        'data': np.random.randn(8),
                                        'weights': np.random.rand(8)})

In [138]:
df

,category,data,weights
0,a,-0.214587,0.933219
1,a,0.209532,0.446363
2,a,-0.609843,0.283683
3,a,-0.078928,0.938450
4,b,-0.107013,0.423512
5,b,0.510684,0.435999
6,b,-1.836560,0.721085
7,b,0.011521,0.786942


In [139]:
grouped = df.groupby('category')

In [141]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

**np.average()=** Belirtilen eksen boyunca ağırlıklı ortalamayı hesaplayın.
Ağırlıklı ortalama, her bileşenin önemini yansıtan bir faktörle çarpılmasından elde edilen bir ortalamadır.

In [142]:
grouped.apply(get_wavg)

category
a   -0.135988
b   -0.480631
dtype: float64

In [143]:
 close_px = pd.read_csv('../examples/stock_px_2.csv', parse_dates=True,
                        index_col=0)
 

In [145]:
close_px.info()
#dataframe.info()işlevi, veri çerçevesinin kısa bir özetini almak için kullanılır. 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [146]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [148]:
spx_corr = lambda x: x.corrwith(x['SPX'])

**corrwith=** İkili korelasyon, Series veya DataFrame satırları veya sütunları olan DataFrame'in satırları veya sütunları arasında hesaplanır. DataFrame'ler, korelasyonları hesaplamadan önce her iki eksen boyunca hizalanır.

In [149]:
 rets = close_px.pct_change().dropna()

In [150]:
get_year = lambda x: x.year

In [151]:
by_year = rets.groupby(get_year)

In [152]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [153]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### **Example: Group-Wise Linear Regression**